In [14]:
import numpy as np
import pandas as pd
import bson
import os,io,sys
from skimage.data import imread
import tensorflow as tf
import threading
print('All modules imported')

All modules imported


In [15]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [16]:
#File paths
cat_file = 'data/category_names.csv'
train_file= 'data/train_example.bson'
train_dir='data/train/'
valid_dir='data/validation/'
test_dir='data/test/'
out_dir=''

In [17]:
categories = pd.read_csv(cat_file)
tot_categories=len(set(categories.category_id))
print('Total Categories :'+ str(tot_categories))

Total Categories :5270


In [18]:
#Load bson training data
data= bson.decode_file_iter(open(train_file,'rb'))

In [19]:
'''
print(data)
for i,d in enumerate(data):
    label=d['category_id']
    id=d['_id']
    img_cnt=len(d['imgs'])
    print(str(label)+'----' +str(id)+'-----'+str(img_cnt))
'''
    
   

"\nprint(data)\nfor i,d in enumerate(data):\n    label=d['category_id']\n    id=d['_id']\n    img_cnt=len(d['imgs'])\n    print(str(label)+'----' +str(id)+'-----'+str(img_cnt))\n"

In [20]:
# Normalize images
def normalize(image_data):
    X= image_data
    Xmin = np.amin(image_data)
    Xmax = np.amax(image_data)
    
    Xscale = (X-Xmin)/(Xmax-Xmin)
    
    return Xscale


In [21]:
queue_input_data = tf.placeholder(tf.float32, shape=[None,180,180, 3], name='image_input')
queue_input_target = tf.placeholder(tf.int64, shape=[None,1],name='category_id_label')

In [22]:
fifo_q_cap=20
batch_size=20
train_batch_cap=20

In [23]:
queue = tf.FIFOQueue(capacity=fifo_q_cap, dtypes=[tf.float32, tf.int64], shapes=[[180,180,3], [1]])

enqueue_op = queue.enqueue_many([queue_input_data, queue_input_target])
dequeue_op = queue.dequeue()

# tensorflow recommendation:
# capacity = min_after_dequeue + (num_threads + a small safety margin) * batch_size
data_batch, target_batch = tf.train.batch(dequeue_op, batch_size=batch_size, capacity=train_batch_cap)


In [24]:
def enqueue(sess,nq_size=20):
    q_data=[]
    q_label=[]
    curr_data=[]
    curr_label=[]
    high=0
    print("starting to write into queue")
    for i,d in enumerate(data):
        label=d['category_id']
        id=d['_id']
        for e,pic in enumerate(d['imgs']):
            picture= imread(io.BytesIO(pic['picture']))
            norm_pic = normalize(picture)
            q_data.append(norm_pic)
            q_label.append([label])
            high+=1
            if high % nq_size==0 : 
                print("Enqueuing upto", high," records")
                curr_data=q_data[0:nq_size]
                curr_label=q_label[0:nq_size]
                #print(str(q_label)+'----' +str(len(q_data)))
                q_data=[]
                q_label=[]
                sess.run(enqueue_op, feed_dict={queue_input_data: curr_data,
                                    queue_input_target: curr_label})

print("finished enqueueing")



finished enqueueing


In [25]:
sess = tf.Session()
nq_size=100
enqueue_thread = threading.Thread(target=enqueue, args=[sess, nq_size])
enqueue_thread.isDaemon()
enqueue_thread.start()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

False

starting to write into queue
Enqueuing upto 100  records


In [26]:
# use this to shuffle batches:
# Fetch the data from the pipeline and put it where it belongs (into your model)
try:
    while not coord.should_stop():
        run_options = tf.RunOptions(timeout_in_ms=4000)
        curr_data_batch, curr_label_batch = sess.run([data_batch, target_batch],options=run_options)
        if coord.should_stop():
            coord.request_stop()
        
except Exception as e:
  # Report exceptions to the coordinator.
    sess.run(queue.close(cancel_pending_enqueues=True))
    coord.request_stop(e)
    
finally:
  # Terminate as usual. It is safe to call `coord.request_stop()` twice.
    sess.run(queue.close(cancel_pending_enqueues=True))
    coord.request_stop()
    coord.join(threads)
  

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.DeadlineExceededError'>, Timed out waiting for notification


DeadlineExceededError: Timed out waiting for notification

In [ ]:
sess.run(queue.close(cancel_pending_enqueues=True))
coord.request_stop()
coord.join(threads)
sess.close()

In [ ]:
# Display Sample image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
print("sample image")
plt.imshow(picture,cm.binary)
plt.show()

In [ ]:
#Image shape
sample_pic= np.array(picture)
sample_pic.shape

In [ ]:
# Normalize images
def normalize(image_data):
    X= image_data
    Xmin = np.amin(image_data)
    Xmax = np.amax(image_data)
    
    Xscale = (X-Xmin)/(Xmax-Xmin)
    
    return Xscale


#Test normalize
#normalized_sample = normalize(sample_pic)
#print(normalized_sample[170][155][1])



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample
one_hot_enc= LabelBinarizer()

labels= one_hot_enc.fit_transform(labels)
print(labels.shape)
print('Labels One-Hot Encoded')

In [ ]:
print(sum([x : x for x in train_cats.values()]))